In [169]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [170]:
df = pd.read_csv('test.csv')
dft = pd.read_csv('todasCenario2.csv')

In [171]:
numero_linhas = df.shape[0]

print(f"O número de linhas no DataFrame é: {numero_linhas}")

O número de linhas no DataFrame é: 3490


In [172]:
#Antes de mudar formatos

# SEATS __________________________________________________
# Check for model cars with missing values in the 'seats' column
distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)

# Update number of seats based on conditions
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI']), 'seats'] = 5
df.loc[df['full_name'] == '2008 Ford Endeavour 3.0 4x4 Thunder Plus', 'seats'] = 7

# Check
distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)


# ENGINE __________________________________________________
#remove cc from engine_capacity
df['engine_capacity'] = df['engine_capacity'].str.replace('cc', '')

# FUEL TYPE __________________________________________________
# unique 'fuel_type'
valores_unicos_fuel_type = df['fuel_type'].unique()
print()
print(valores_unicos_fuel_type)

# KMS __________________________________________________
#remove kms from kms_driven
df['kms_driven'] = df['kms_driven'].str.replace('Kms', '')
# Remove , and blanks from 'kms_driven'
df['kms_driven'] = df['kms_driven'].str.replace(',', '').str.strip()

# MAX POWER 
# #convert to bhp__________________________________________________
Correct_Max_Power = []
for power in df['max_power']:
    if pd.notna(power):
        if isinstance(power, str):
            if 'bhp' in power:
                num_bhp = float(re.search(r'\d+.?\d', power).group())
                Correct_Max_Power.append(num_bhp)
            elif 'PS' in power:
                num_ps = float(re.search(r'\d+.?\d', power).group())
                num_bhp = round(num_ps * 0.98632, 2)
                Correct_Max_Power.append(num_bhp)
            else:
                num = re.search(r'\d+\.?\d*', power)
                Correct_Max_Power.append(float(num.group()) if num else None)
        else:
            Correct_Max_Power.append(power)
    else:
        Correct_Max_Power.append(None)


# Convert not NaN to float
Correct_Max_Power = [float(value) if value is not None else value for value in Correct_Max_Power]

df['max_power'] = Correct_Max_Power

# OWNER TYPE __________________________________________________
# Delet blank spaces in 'owner_type'
df['owner_type'].fillna(0, inplace=True)


# Check
unique_values = df['owner_type'].unique()
print()
print(unique_values)



# INSURANCE __________________________________________________
#Delet "insurance" 
df['insurance'] = df['insurance'].str.replace('insurance', '')

#Assign value to blank lines
df['insurance'].fillna('No Insurance', inplace=True)

#Blank space on "Third Party" lines
df['insurance'] = df['insurance'].str.replace('Third Party ', 'Third Party')

#Remove integer values
df = df[~df['insurance'].astype(str).str.isnumeric()]

#MILEAGE
#assign value to blank lines
df['mileage'] = df['mileage'].fillna(0)

df = df[df['mileage'].notna()]
consumo_correto = []

for i in df['mileage']:
    match = re.search(r'(\d+.\d+|\d+)', str(i))  # Encontrar números (com ou sem ponto decimal)

    if match:
        valor = float(match.group())

        if 'km/kg' in str(i):
            valor *= 1.40  # Converter de km/kg para kmpl

        consumo_correto.append(valor)

df['mileage'] = consumo_correto





Distinct full names with blank seats:
2005 Honda City 1.5 EXI
2009 Maruti Estilo LXI
2012 Honda Jazz Select Edition

Distinct full names with blank seats:
2005 Honda City 1.5 EXI
2012 Honda Jazz Select Edition

['Petrol' 'Diesel' 'CNG' 'Electric' 'LPG']

['First Owner' 'Second Owner' 'Third Owner' 0 'Fourth Owner' 'Fifth Owner']


In [173]:
numero_linhas = df.shape[0]

print(f"O número de linhas no DataFrame é: {numero_linhas}")

O número de linhas no DataFrame é: 3490


In [174]:
#FullName ______________________________________

#ANO
# Extract the year and create a new column
df['ano'] = df['full_name'].str.extract(r'(\d{4})')

# Display the resulting DataFrame
print(df['ano'])

#MARCA
# Extract the first word after the year and create a new column
df['marca'] = df['full_name'].str.extract(r'\d{4}\s(\w+)')

# Print the resulting DataFrame
print(df['marca'])

#MODELO
# Get everything that comes after "marca"
df['modelo'] = df['full_name'].str.extract(r'\d{4}\s\w+\s(.+)')

# Print the resulting DataFrame
print(df['modelo'])

0       2017
1       2020
2       2010
3       2010
4       2018
        ... 
3485    2022
3486    2017
3487    2017
3488    2016
3489    2017
Name: ano, Length: 3490, dtype: object
0       Hyundai
1        Maruti
2        Maruti
3        Maruti
4       Hyundai
         ...   
3485       Tata
3486    Renault
3487      Honda
3488      Honda
3489     Maruti
Name: marca, Length: 3490, dtype: object
0       Grand i10 1.2 Kappa Sportz Option
1                           Swift AMT VXI
2                     Zen Estilo LX BSIII
3                                Alto LXi
4                     i20 Active S Petrol
                      ...                
3485                     Tigor EV XZ Plus
3486                         KWID 1.0 RXT
3487                       City i VTEC VX
3488                    Jazz 1.2 V i VTEC
3489                            Swift VXI
Name: modelo, Length: 3490, dtype: object


In [175]:
#BODY TYPE ----------------------
#Replace the missing types and the ones that were wrong
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI', '2019 Maruti Wagon R ZXI AMT 1.2BSIV',
                             '2016 Mercedes-Benz B Class B180 Sport', '2016 Mercedes-Benz A Class A200 D Sport Edition',
                             '2004 Tata Indigo Marina GLX', '2017 Volvo V40 D3 R Design']), 'body_type'] = 'Hatchback'
df.loc[df['full_name'].isin(['2005 Tata Sumo Victa EX 10/7 Str BSII', '2008 Tata Sumo CX', '2010 Tata New Safari DICOR 2.2 EX 4x2', 
                             '2017 Mahindra KUV 100 mFALCON G80 K8', '2019 Mahindra Bolero Camper GOLD ZX 2WD']), 'body_type'] = 'SUV'
df.loc[df['full_name'].isin(['2015 Honda Mobilio E i-DTEC', '2018 Maruti Eeco 5 Seater AC BSIV', '2016 Datsun GO Plus A']), 'body_type'] = 'MUV'
df.loc[df['full_name'].isin(['2021 BMW 7 Series 730Ld DPE Signature','2006 Hyundai Accent CRDi', '2007 Maruti SX4 Vxi BSIII',
                             '2020 Mercedes-Benz S-Class Maybach S560', '2004 Skoda Octavia 1.8 Turbo Petrol RS MT']), 'body_type'] = 'Sedan'
df.loc[df['full_name'].isin(['2012 Mercedes-Benz SLK-Class SLK 350', '2013 Mercedes-Benz E-Class E350 Cabriolet', 
                             '2009 Mercedes-Benz SLK-Class 55 AMG']), 'body_type'] = 'Convertibles'
df.loc[df['full_name'] == '2011 Jaguar XK R V8 Coupe', 'body_type'] = 'Coupe'
df.loc[df['full_name'].isin(['2015 Chevrolet Enjoy 1.4 LS 7', '2004 Maruti Versa DX2 8-SEATER BSIII TWIN A/C', '2021 Maruti Eeco 5 STR With AC Plus HTR CNG',
                             '2022 Maruti Eeco 5 Seater AC 2020-2022', '2022 Maruti Eeco 5 Seater AC BSVI', '2017 Maruti Eeco Flexi Green', 
                             '2013 Maruti Omni Limited Edition', '2021 Maruti Eeco CNG 5 Seater AC', '2019 Maruti Eeco CNG 5 Seater AC BSIV', 
                             '2019 Maruti Eeco 5 Seater AC', '2013 Maruti Eeco 5 Seater Standard BSIV', '2015 Maruti Omni CNG', 
                             '2015 Maruti Omni MPI CARGO BSIV', '2016 Maruti Eeco CNG 5 Seater AC BSIV', '2013 Toyota Innova 2.5 E Diesel MS 7-seater',
                             '2015 Chevrolet Enjoy 1.4 LS 7']), 'body_type'] = 'Minivans'
df.loc[df['full_name'].isin(['2014 Tata Xenon XT EX 4X2', '2019 Isuzu D-Max High']), 'body_type'] = 'Pickup'

# Check
distinct_full_names_with_blank_body_type = df[df['body_type'].isnull()]['full_name'].unique()
print('Distinct full names with blank body_type:')
for full_name in distinct_full_names_with_blank_body_type:
    print(full_name)

Distinct full names with blank body_type:


In [176]:
# Mudar formatos

df = df.astype({
    'seats': float,
    'engine_capacity': float,
    'kms_driven': float,
    'max_power': float,
    'ano': float,
    'marca': 'string',
    'modelo': 'string',
    'body_type': 'string',
    'mileage': float,
})

In [177]:
# Importe a biblioteca pandas, se ainda não estiver importada
import pandas as pd

# Substitua os valores NaN por 'Indefinido' nas colunas específicas, exceto 'engine_capacity'
columns_to_replace = ['kms_driven', 'marca', 'modelo']
df[columns_to_replace] = df[columns_to_replace].fillna('Indefinido')

# Substitua os valores NaN na coluna 'engine_capacity' por 0
df['engine_capacity'] = df['engine_capacity'].fillna(0)
# Substitua os valores NaN na coluna 'max_power' por 0
df['max_power'] = df['max_power'].fillna(0)

In [178]:
# Change formats

df = df.astype({
    'seats': float,
    'engine_capacity': float,
    'kms_driven': float,
    'max_power': float,
    'ano': float,
    'marca': 'string',
    'modelo': 'string',
   
})

In [179]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['max_power'], dft['max_power_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['max_power_group'] = df['max_power'].map(mapeamento_max_power_group)

# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['max_power_group'], dft['id_max_power_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_max_power_group'] = df['max_power_group'].map(mapeamento_max_power_group)

In [180]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_max_power_group'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_max_power_group'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado

In [181]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['marca'], dft['categoria_marca']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['categoria_marca'] = df['marca'].map(mapeamento_max_power_group)

# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['categoria_marca'], dft['id_categoria_marca']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_categoria_marca'] = df['categoria_marca'].map(mapeamento_max_power_group)

In [182]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_categoria_marca'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_categoria_marca'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado

In [183]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['kms_driven'], dft['kms_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['kms_group'] = df['kms_driven'].map(mapeamento_max_power_group)

# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['kms_group'], dft['id_kms_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_kms_group'] = df['kms_group'].map(mapeamento_max_power_group)

In [184]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_kms_group'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_kms_group'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado

In [185]:
# groups engine_capacity

limites = [0,1, 1000, 2000, 3000, 4000, float('inf')]  
rotulos = ['Elétrico', 'até 1000 cc', 'até 2000 cc', 'até 3000 cc', 'até 4000 cc', 'mais de 4000 cc']

df['engine_capacity_group'] = pd.cut(df['engine_capacity'], bins=limites, labels=rotulos, right=False)

# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['engine_capacity_group'], dft['id_engine_capacity_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_engine_capacity_group'] = df['engine_capacity_group'].map(mapeamento_max_power_group)

In [186]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_engine_capacity_group'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 0.1

    # Converter a coluna 'id_engine_capacity_group' para tipo numérico (float)
    df['id_engine_capacity_group'] = df['id_engine_capacity_group'].astype(float)

    # Substituir os valores NaN na coluna 'id_engine_capacity_group' em df pelo novo valor
    df['id_engine_capacity_group'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df.head())  # Exemplo de visualização, pode variar conforme a estrutura do DataFrame


                                        full_name registered_year  \
0  2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                       2020 Maruti Swift AMT VXI        Feb 2021   
2                 2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                            2010 Maruti Alto LXi            2010   
4                2018 Hyundai i20 Active S Petrol        Oct 2018   

   engine_capacity      insurance transmission_type  kms_driven    owner_type  \
0           1197.0  Comprehensive            Manual     53658.0   First Owner   
1           1197.0  Not Available         Automatic     33055.0  Second Owner   
2            998.0    Third Party            Manual     90385.0   Third Owner   
3            796.0    Third Party            Manual    110000.0   First Owner   
4           1197.0  Comprehensive            Manual     58000.0  Second Owner   

  fuel_type  max_power  seats  ...    marca  \
0    Petrol       81.8    5.0  ...  Hyundai   
1   

In [187]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['body_type'], dft['id_body_type']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_body_type'] = df['body_type'].map(mapeamento_max_power_group)

In [188]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_body_type'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_body_type'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado

In [189]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['city'], dft['id_city']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_city'] = df['city'].map(mapeamento_max_power_group)

In [190]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['insurance'], dft['id_insurance']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_insurance'] = df['insurance'].map(mapeamento_max_power_group)

In [191]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['modelo'], dft['id_modelos']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_modelos'] = df['modelo'].map(mapeamento_max_power_group)

In [192]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['fuel_type'], dft['id_fuel_type']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_fuel_type'] = df['fuel_type'].map(mapeamento_max_power_group)

In [193]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['owner_type'], dft['id_owner_type']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_owner_type'] = df['owner_type'].map(mapeamento_max_power_group)

In [194]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['transmission_type'], dft['id_transmissao']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_transmissao'] = df['transmission_type'].map(mapeamento_max_power_group)

In [195]:
df = df.astype({
    
    'ano': float,
  
    
  
})

In [196]:
contagem_nulos = df.isnull().sum()

print(f"O número de valores nulos na coluna é: {contagem_nulos}")

O número de valores nulos na coluna é: full_name                      0
registered_year               15
engine_capacity                0
insurance                      0
transmission_type              0
kms_driven                     0
owner_type                     0
fuel_type                      0
max_power                      0
seats                          2
mileage                        0
body_type                      0
city                           0
ID                             0
ano                            0
marca                          0
modelo                         0
max_power_group               69
id_max_power_group             0
categoria_marca               73
id_categoria_marca             0
kms_group                   1609
id_kms_group                   0
engine_capacity_group          0
id_engine_capacity_group       0
id_body_type                   0
id_city                        0
id_insurance                   0
id_modelos                   396
id_f

In [197]:
numero_linhas = df.shape[0]

print(f"O número de linhas no DataFrame é: {numero_linhas}")

O número de linhas no DataFrame é: 3490


In [198]:
df

,full_name,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,...,id_kms_group,engine_capacity_group,id_engine_capacity_group,id_body_type,id_city,id_insurance,id_modelos,id_fuel_type,id_owner_type,id_transmissao
0,2017 Hyundai Grand i10 1.2 Kappa Sportz Option,Jun 2017,1197.0,Comprehensive,Manual,53658.0,First Owner,Petrol,81.8,5.0,...,3.750793,até 2000 cc,0.126137,0.364112,0.922710,0.445408,2.872642,0.180334,0.150465,0.131359
1,2020 Maruti Swift AMT VXI,Feb 2021,1197.0,Not Available,Automatic,33055.0,Second Owner,Petrol,81.8,5.0,...,3.750793,até 2000 cc,0.126137,0.364112,1.281577,1.418644,3.224824,0.180334,0.620328,0.583102
2,2010 Maruti Zen Estilo LX BSIII,Jul 2010,998.0,Third Party,Manual,90385.0,Third Owner,Petrol,67.1,5.0,...,3.750793,até 1000 cc,0.776698,0.364112,1.593888,0.256633,3.428944,0.180334,1.365236,0.131359
3,2010 Maruti Alto LXi,2010,796.0,Third Party,Manual,110000.0,First Owner,Petrol,46.3,5.0,...,0.255059,até 1000 cc,0.776698,0.364112,0.767700,0.256633,2.420344,0.180334,0.150465,0.131359
4,2018 Hyundai i20 Active S Petrol,Oct 2018,1197.0,Comprehensive,Manual,58000.0,Second Owner,Petrol,81.8,5.0,...,0.255059,até 2000 cc,0.126137,0.364112,0.767700,0.445408,3.826884,0.180334,0.620328,0.131359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,2022 Tata Tigor EV XZ Plus,2022,0.0,Third Party,Automatic,30000.0,First Owner,Electric,73.7,5.0,...,0.364786,Elétrico,2.536850,0.578053,1.096102,0.256633,3.428944,2.455816,0.150465,0.583102
3486,2017 Renault KWID 1.0 RXT,Apr 2017,999.0,Third Party,Manual,64024.0,First Owner,Petrol,67.0,5.0,...,3.750793,até 1000 cc,0.776698,0.364112,0.868320,0.256633,2.650793,0.180334,0.150465,0.131359
3487,2017 Honda City i VTEC VX,Dec 2017,1497.0,Comprehensive,Manual,39000.0,First Owner,Petrol,117.3,5.0,...,0.364786,até 2000 cc,0.126137,0.578053,0.767700,0.445408,2.729974,0.180334,0.150465,0.131359
3488,2016 Honda Jazz 1.2 V i VTEC,Jun 2016,1199.0,Comprehensive,Manual,33000.0,First Owner,Petrol,88.7,5.0,...,0.364786,até 2000 cc,0.126137,0.364112,0.922710,0.445408,2.872642,0.180334,0.150465,0.131359


In [199]:
df.columns

Index(['full_name', 'registered_year', 'engine_capacity', 'insurance',
       'transmission_type', 'kms_driven', 'owner_type', 'fuel_type',
       'max_power', 'seats', 'mileage', 'body_type', 'city', 'ID', 'ano',
       'marca', 'modelo', 'max_power_group', 'id_max_power_group',
       'categoria_marca', 'id_categoria_marca', 'kms_group', 'id_kms_group',
       'engine_capacity_group', 'id_engine_capacity_group', 'id_body_type',
       'id_city', 'id_insurance', 'id_modelos', 'id_fuel_type',
       'id_owner_type', 'id_transmissao'],
      dtype='object')

In [200]:
#Export
#Not the target
colunas_selecionadas = ['id_body_type','id_modelos','id_categoria_marca','ano','id_city','id_insurance', 'id_transmissao','seats','mileage','id_engine_capacity_group','id_fuel_type','id_kms_group','id_max_power_group','id_owner_type', 'ID']


if set(colunas_selecionadas).issubset(df.columns):
    df_selecionado = df[colunas_selecionadas]
    df_selecionado.to_csv('TestCenario2.csv', index=False)